In [230]:
import tensorflow as tf                              # 텐서플로 호출
from tensorflow.keras.preprocessing import image     # 케라스 이미지 호출
import numpy as np                                   # 넘파이 호출                               
import matplotlib.pyplot as plt                      # 파이플랏
from tensorflow.image import pad_to_bounding_box     # 이미지 묶기
from tensorflow.image import central_crop            # 가운데 추출
from tensorflow.image import resize                  # 이미지 크기조절
from PIL import Image
from sklearn.model_selection import train_test_split

In [231]:
image_arrays = []  # 이미지 배열을 저장할 리스트      
for i in range(200):
    # i를 3자리 숫자로 포맷팅하여 파일 이름을 생성
    file_name = f'tf_image/train_{i:03d}.jpg'
    img = image.load_img(file_name)
    image_arrays.append(image.img_to_array(img).astype('float32') / 255)

In [232]:
test_arrays = []  # 이미지 배열을 저장할 리스트      
for i in range(60):
    # i를 3자리 숫자로 포맷팅하여 파일 이름을 생성
    file_name = f'tf_image/test_{i:03d}.jpg'
    img = image.load_img(file_name)
    test_arrays.append(image.img_to_array(img).astype('float32') / 255)

In [233]:
image_zunsik = []
for i in range(len(image_arrays)):
    flip_lr_tensor = tf.image.flip_left_right(image_arrays[i])
    flip_ud_tensor = tf.image.flip_up_down(image_arrays[i]) 
    flip_lr_ud_tensor = tf.image.flip_up_down(flip_lr_tensor) 
    flip_or_image = tf.keras.preprocessing.image.array_to_img(image_arrays[i])
    flip_lr_image = tf.keras.preprocessing.image.array_to_img(flip_lr_tensor)
    flip_ud_image = tf.keras.preprocessing.image.array_to_img(flip_ud_tensor)
    flip_lr_ud_image = tf.keras.preprocessing.image.array_to_img(flip_lr_ud_tensor)
    listy1 = [flip_or_image,flip_lr_image,flip_ud_image,flip_lr_ud_image]
    listy1 = [flip_or_image,flip_lr_image,flip_ud_image,flip_lr_ud_image]
    for j in listy1:
        for k in range(2):
            random_bright_tensor = tf.image.random_brightness(j, max_delta=0.9) 
            # random_bright_tensor = tf.clip_by_value(random_bright_tensor, 0, 1)
            # random_bright_image = tf.keras.preprocessing.image.array_to_img(random_bright_tensor)
            image_zunsik.append(random_bright_tensor)
len(image_zunsik)

1600

In [234]:
train_set = np.array(image_zunsik).astype('float32')/255

In [235]:
train_ans_list = []
for m in range(20):
    for n in range(80):
        train_ans_list.append(m)
ans_set = np.array(train_ans_list)

In [236]:
X_train, X_test, y_train, y_test = train_test_split(train_set, ans_set, test_size=0.2)
y_train = to_categorical(y_train, 20)
y_test = to_categorical(y_test, 20)


In [274]:
real_ans = []
for m in range(20):
    for n in range(3):
        real_ans.append(m)
real_ans = to_categorical(real_ans, 20)
real_ans = np.array(real_ans)

In [272]:
real_test = np.array(test_arrays)

In [237]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D # 랜덤 히든계층, 플래튼, 콘브, 맥스풀 이렇게 된다. 
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping 
from tensorflow.keras.datasets import mnist        
from tensorflow.keras.utils import to_categorical

import matplotlib.pyplot as plt
import numpy as np

In [263]:
model = Sequential()
model.add(Conv2D(128, kernel_size=(5,5), input_shape=(150, 500, 3), activation='relu')) # 필터갯수, 필터크기, 투입사진, 작동방식, 스트라이드 설정
model.add(Conv2D(64, (3, 3), activation='relu'))      # 필터갯수, 필터크기(생략되었다), 작동방식
model.add(MaxPooling2D(pool_size=(4,4)))              # 풀링하기.2,2사이즈로 만들기                        # 25%를 드롭아웃한 상태에서 진행
model.add(Flatten())                                  # 플래튼이 일어난 다음에 히든계층으로 간다. 
model.add(Dense(32,  activation='relu'))             # 128차원짜리 층으로 나눔
model.add(Dropout(0.3))                               # 50% 드롭
model.add(Dense(20, activation='softmax'))            # 소프트맥스로 결과값 출력

In [264]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [265]:
MODEL_DIR = './data/model/'

In [266]:
modelpath="./data/model/MNIST_MLP.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=0, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

In [267]:
history = model.fit(X_train,y_train , validation_split=0.25, epochs=30, batch_size=20, verbose=1, callbacks=[early_stopping_callback,checkpointer])

Epoch 1/30
48/48 [==============================] - 149s 3s/step - loss: 3.3909 - accuracy: 0.0760 - val_loss: 2.9156 - val_accuracy: 0.1063
Epoch 2/30


C:\Users\user\anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


48/48 [==============================] - 148s 3s/step - loss: 2.5561 - accuracy: 0.2125 - val_loss: 2.2657 - val_accuracy: 0.4219
Epoch 3/30
48/48 [==============================] - 148s 3s/step - loss: 1.9186 - accuracy: 0.4635 - val_loss: 2.2324 - val_accuracy: 0.4313
Epoch 4/30
48/48 [==============================] - 148s 3s/step - loss: 1.7718 - accuracy: 0.4885 - val_loss: 2.0871 - val_accuracy: 0.4500
Epoch 5/30
48/48 [==============================] - 148s 3s/step - loss: 1.3975 - accuracy: 0.6177 - val_loss: 1.7137 - val_accuracy: 0.6250
Epoch 6/30
48/48 [==============================] - 148s 3s/step - loss: 1.2226 - accuracy: 0.6677 - val_loss: 1.7195 - val_accuracy: 0.6187
Epoch 7/30
48/48 [==============================] - 148s 3s/step - loss: 1.0753 - accuracy: 0.7104 - val_loss: 1.4548 - val_accuracy: 0.7437
Epoch 8/30
48/48 [==============================] - 149s 3s/step - loss: 1.0015 - accuracy: 0.7542 - val_loss: 1.7199 - val_accuracy: 0.6000
Epoch 9/30
48/48 [======

In [275]:
print("\n Test Accuracy: %.4f" % (model.evaluate(real_test, real_ans)[1]))

2/2 [==============================] - 2s 852ms/step - loss: 9.6345 - accuracy: 0.3833

 Test Accuracy: 0.3833
